
# 根据上下文长度选择LLM

不同的LLM具有不同的上下文长度。作为一个非常直接和实际的例子，OpenAI有两个版本的GPT-3.5-Turbo：一个上下文长度为4k，另一个上下文长度为16k。本笔记展示了如何根据输入在它们之间进行路由。


In [24]:
# 导入所需模块
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompt_values import PromptValue
from langchain_openai import ChatOpenAI

In [3]:
# 导入ChatOpenAI类
from openai import ChatOpenAI

# 创建一个使用gpt-3.5-turbo模型的short_context_model对象
short_context_model = ChatOpenAI(model="gpt-3.5-turbo")

# 创建一个使用gpt-3.5-turbo-16k模型的long_context_model对象
long_context_model = ChatOpenAI(model="gpt-3.5-turbo-16k")

In [4]:
def get_context_length(prompt: PromptValue):
    # 将PromptValue对象转换为messages列表
    messages = prompt.to_messages()
    # 通过short_context_model获取messages中的token数量
    tokens = short_context_model.get_num_tokens_from_messages(messages)
    # 返回tokens数量
    return tokens


In [5]:


# 定义一个变量prompt，使用PromptTemplate类的from_template方法，传入一个字符串作为参数
# 这个字符串是一个模板，包含了一个占位符{context}，用于接收后续传入的具体内容
prompt = PromptTemplate.from_template("Summarize this passage: {context}")

In [20]:
def choose_model(prompt: PromptValue):
    context_len = get_context_length(prompt)  # 获取上下文长度
    if context_len < 30:  # 如果上下文长度小于30
        print("short model")  # 打印"短模型"
        return short_context_model  # 返回短上下文模型
    else:
        print("long model")  # 打印"长模型"
        return long_context_model  # 返回长上下文模型


In [25]:
# 创建一个链式操作对象chain
chain = prompt | choose_model | StrOutputParser()

# prompt是一个输入对象，用于提供输入文本
# choose_model是一个模型选择对象，用于选择适合的模型
# StrOutputParser是一个字符串输出解析对象，用于解析模型的输出结果

# 这行代码将prompt对象传递给choose_model对象，再将choose_model对象的输出传递给StrOutputParser对象
# 最终得到一个可以解析模型输出结果的链式操作对象chain

In [26]:

# 调用chain模块的invoke函数，并传入一个字典作为参数
# 字典中包含一个键值对，键为"context"，值为"a frog went to a pond"
chain.invoke({"context": "a frog went to a pond"})



short model


'The passage mentions that a frog visited a pond.'

In [27]:

# 调用chain的invoke方法，并传入一个字典作为参数
chain.invoke(
    {"context": "a frog went to a pond and sat on a log and went to a different pond"}
)

long model


'The passage describes a frog that moved from one pond to another and perched on a log.'